In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers~=4.15.0
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 3.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.1
    Uninstalling setuptools-67.6.1:
      Successfully uninstalled setuptools-67.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")


Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial_dataset downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial_dataset/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
updated_dataset = dataset.map(lambda examples: {"tag": "Entailment"  if (examples["original_response"] is None and "Hallucination" not in examples["BEGIN"]) else "Hallucination"})

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
counter = 0
for data in updated_dataset["train"]:
  if data["tag"] == "Hallucination":
    counter+=1
print(counter)

15624


In [ ]:
counter/len(updated_dataset["train"])

0.8511194639647001

In [ ]:
ud = updated_dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
ud2 = ud.map(lambda data:{"history":""+(data["knowledge"]) if data["history"] is None else data["history"][-1]+"</s>"+data["knowledge"]})
ud2["train"][3]
# ud2 = ud.map(lambda data:{"history":""+(data["knowledge"]) if data["history"] is None else "</s>".join(data["history"])+"</s>"+data["knowledge"]})
# ud2["train"][3]

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

{'dialog_idx': 0,
 'response': 'Well, I know that it is defined as the ability to connect and use the internet',
 'original_response': 'What is your favorite thing to do with internet access? I like being able to use my computer and smartphone to use my email and browse the world wide web',
 'history': 'Very Interesting. What else can you tell me about Internet Access ?</s>Internet access is the ability of individuals and organizations to connect to the Internet using computer terminals, computers, and other devices; and to access services such as email and the World Wide Web.',
 'knowledge': 'Internet access is the ability of individuals and organizations to connect to the Internet using computer terminals, computers, and other devices; and to access services such as email and the World Wide Web.',
 'BEGIN': ['Hallucination', 'Entailment'],
 'VRM': ['Disclosure', 'Question'],
 'tag': 'Hallucination'}

In [ ]:

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 1024)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    encoding=tokenizer(examples["history"], examples["original_response"], truncation=True)
    lable= 1 if examples['tag'] == "Hallucination" else 0
    encoding["labels"]=lable
    
    return encoding
encoded_dataset = ud2.map(preprocess_function,remove_columns=ud['train'].column_names)

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
id2label = { 1: 'Hallucination', 0: 'Entailment'}
label2id = {'Hallucination': 1, 'Entailment': 0}

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label2id),label2id=label2id,id2label=id2label)


# print(model)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
import evaluate
import numpy as np
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    adafactor= False,
adam_epsilon= 1.0e-08,
    logging_dir='./logs',
    evaluation_strategy='steps',
    save_total_limit=1,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(encoded_dataset['test'])
print(results)

***** Running training *****
  Num examples = 18357
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3444


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.445100,0.432829,0.847820,0.917643
1000,0.423000,0.428418,0.847820,0.917643
1500,0.417000,0.426110,0.847820,0.917643
2000,0.431400,0.426482,0.847820,0.917643
2500,0.412000,0.430308,0.847820,0.917643
3000,0.430900,0.427505,0.847820,0.917643


***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/con

{'eval_loss': 0.471011221408844, 'eval_accuracy': 0.8262220966374683, 'eval_f1': 0.9048429521893858, 'eval_runtime': 24.472, 'eval_samples_per_second': 144.614, 'eval_steps_per_second': 2.288, 'epoch': 3.0}


In [ ]:
results_pred = trainer.predict(encoded_dataset['test'])


***** Running Prediction *****
  Num examples = 3539
  Batch size = 64


In [ ]:
logits = results_pred.predictions


In [ ]:
import tensorflow as tf

In [ ]:
transformed = tf.nn.softmax(logits)

In [ ]:
transformed[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.12654774, 0.8734522 ],
       [0.12654829, 0.87345165],
       [0.12654889, 0.8734511 ],
       [0.12654853, 0.8734515 ],
       [0.12654796, 0.873452  ]], dtype=float32)>

In [ ]:
ground_truth = (encoded_dataset["test"]['labels'])

In [ ]:
transformed_labels = np.argmax(transformed, axis=1)

In [ ]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
f1 = metric_f1.compute(predictions=transformed_labels, references=ground_truth)

# # Load recall metric
metric_recall = evaluate.load("recall")


metric_accuracy = evaluate.load("accuracy")
accu = metric_accuracy.compute(predictions=transformed_labels, references=ground_truth)

# # Compute recall metric
 = metric_recall.compute(predictions=transformed_labels, references=ground_truth)

In [ ]:
f1

{'f1': 0.9048429521893858}

In [ ]:

accu

{'accuracy': 0.8262220966374683}

In [ ]:
recall_score

{'recall': 1.0}

In [ ]:
counter = 0
for label in transformed_labels:
  if label == 0:
    counter +=1
print(counter)

3539
